In [ ]:
%matplotlib notebook

In [ ]:
%matplotlib inline

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import os
import sys
import importlib
from IPython.display import clear_output
from astropy.time import Time

In [ ]:
sys.path.append("/Users/jrobinson/atlas-phase-curves/atlas-phase-curves")
from calculate_phase import atlas_SQL_query_df
from calculate_phase import atlas_database_connection as adb
from calculate_phase import solar_apparitions as sa

In [ ]:
importlib.reload(atlas_SQL_query_df)

In [ ]:
cnx=adb.database_connection().connect()
cursor = cnx.cursor()

In [ ]:
qry = "SHOW TABLES;"
pd.read_sql_query(qry,cnx)

In [ ]:
qry = """select *
    from INFORMATION_SCHEMA.COLUMNS
    where TABLE_NAME='atlas_exposures'"""
pd.read_sql_query(qry,cnx)

In [ ]:
# qry = """SELECT * FROM atlas_exposures"""
# df_exp = pd.read_sql_query(qry,cnx)
# df_exp

In [ ]:
# t = Time.now()
# fname = "atlas_exposures_{:.4f}.csv".format(t.mjd)
# df_exp.to_csv(fname)

In [ ]:
df_exp = pd.read_csv("atlas_exposures_59788.3936.csv", index_col = 0)

In [ ]:
mask_null = np.isnan(df_exp["limiting_magnitude"])
df_lim = df_exp[~mask_null]
df_lim

In [ ]:
df_atlas = pd.read_csv("atlas_co_exposures.tst", delim_whitespace = True)
df_atlas

In [ ]:
df_atlas[df_atlas["expname"]=="01a58016o0492o"].iloc[0]

In [ ]:
# np.isin is very slow for large lists of strings
# set up a subset of each dataframe to compare

_df1=df_lim[["expname","limiting_magnitude"]].copy()

_df2=df_atlas[["expname","mag5sig"]].copy()

# _df1 = _df1.iloc[:10000]
# _df2 = _df2.iloc[:10000]

# create a dataframe that contains all unique names, and the dataframe that they occur in
df_merge = _df1.merge(_df2,on="expname")
df_merge["mjd"] = [int(x[3:8]) for x in df_merge["expname"]]
df_merge

In [ ]:
x_plot = "mag5sig"
y_plot = "limiting_magnitude"
c_plot = "mjd"
df_plot = df_merge


fig = plt.figure()
gs = gridspec.GridSpec(1,1)
ax1 = plt.subplot(gs[0,0])


s1 = ax1.scatter(df_plot[x_plot],df_plot[y_plot], c = df_plot[c_plot])
cbar1 = fig.colorbar(s1)
cbar1.set_label(c_plot)

ax1.plot([np.amin(df_plot[x_plot]),np.amax(df_plot[x_plot])],
        [np.amin(df_plot[x_plot]),np.amax(df_plot[x_plot])], c="r")

ax1.set_xlabel(x_plot)
ax1.set_ylabel(y_plot)

plt.show()

In [ ]:
x_plot = "mjd"
y_plot1 = "mag5sig"
y_plot2 = "limiting_magnitude"
df_plot = df_merge

fig = plt.figure()
gs = gridspec.GridSpec(1,1)
ax1 = plt.subplot(gs[0,0])

s1 = ax1.scatter(df_plot[x_plot],df_plot[y_plot2]-df_plot[y_plot1],s=1)

ax1.axvline(57855,c="k")

ax1.set_xlabel(x_plot)
ax1.set_ylabel("{}-{}".format(y_plot2,y_plot1))

plt.show()

In [ ]:
# qry = """SELECT primaryId,dateCreated,dateLastModified,mjd,expname FROM atlas_exposures WHERE limiting_magnitude is null"""
# df_null = pd.read_sql_query(qry,cnx)
df_null = df_exp[mask_null]
df_null

In [ ]:
df_null[df_null["expname"]=="01a57844o0397o"]

In [ ]:
df_atlas[df_atlas["expname"]=="01a57844o0397o"]

In [ ]:
# for i in np.arange(391125,400000,1):
#     try:
#         print(df_null.loc[i])
#         break
#     except:
#         continue

In [ ]:
df_atlas[df_atlas["expname"]=="02a59639o0244c"].iloc[0]["mag5sig"]

In [ ]:
# reselect the exposures where limiting magnitude is null when rerun
# This way we don't reupdate exposures which are already fixed
qry = """SELECT expname FROM atlas_exposures where limiting_magnitude is null"""
df_null = pd.read_sql_query(qry,cnx)
df_null

In [ ]:
len(df_atlas)

In [ ]:
# combine the two sources of exposure names
_df1=df_null[["expname"]].copy()
_df1["df"] = "rockAtlas NULL"

_df2=df_atlas[["expname"]].copy()
_df2["df"] = "atlas"
              
_df = pd.concat([_df1,_df2])
_df

In [ ]:
# keep only the rows that are duplicates, i.e. the exposure names that are in null list AND present in df_atlas
_df_search = _df[_df.duplicated(['expname'], keep=False)]
_df_search

In [ ]:
_df_search[_df_search["expname"]=="01a57800o0484o"]

In [ ]:
df_search = _df_search[_df_search["df"]=="rockAtlas NULL"]
df_search

In [ ]:
df_atlas[df_atlas["expname"]=="01a57851o0638o"]

In [ ]:
df = df_search

for i in range(len(df)):
        
#     if i>20:
#         break
            
    expname = df.iloc[i]["expname"]
    
    try:
        mag5sig = df_atlas[df_atlas["expname"]==expname].iloc[0]["mag5sig"]
    except:
        print("{}, {} mag5sig not found".format(i,expname))
        continue

    qry = """UPDATE atlas_exposures
            SET limiting_magnitude={}
            WHERE expname='{}'""".format(mag5sig,expname)

    print(i, qry)

    cursor.execute(qry)
    cnx.commit()
